# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,luderitz,-26.6481,15.1594,20.94,41,0,6.02,NaN,1695838990
1,1,puerto natales,-51.7236,-72.4875,3.25,81,75,2.06,CL,1695838990
2,2,caluula,11.9661,50.7569,28.18,72,39,1.49,SO,1695838990
3,3,tempoal de sanchez,21.5167,-98.3833,36.73,32,1,1.64,MX,1695838991
4,4,hamilton,39.1834,-84.5333,21.61,92,100,2.57,US,1695838948


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_destinations_df = city_data_df.loc[(city_data_df["Max Temp"].astype(float) <= 25)
                                         & (city_data_df["Max Temp"].astype(float) >= 15)
                                         & (city_data_df["Humidity"].astype(float) <= 70)
                                         & (city_data_df["Humidity"].astype(float) >= 30)
                                         & (city_data_df["Cloudiness"].astype(float) >= 10)
                                         & (city_data_df["Cloudiness"].astype(float) <= 50)]

# Drop any rows with null values
ideal_destinations_df.dropna()

# Display sample data
ideal_destinations_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,fallon,39.4735,-118.7774,22.18,37,40,1.54,US,1695838731
40,40,villa gesell,-37.2639,-56.9730,15.59,59,10,4.37,AR,1695838995
45,45,kapuskasing,49.4169,-82.4331,19.05,42,20,5.14,CA,1695838996
51,51,constantia,44.1833,28.6500,20.39,65,33,9.31,RO,1695838997
81,81,guerrero negro,27.9769,-114.0611,23.71,61,49,6.17,MX,1695839001
104,104,kununurra,-15.7667,128.7333,23.99,60,38,3.60,AU,1695839005
137,137,dryden,49.7833,-92.7503,18.16,68,40,6.17,CA,1695839011
180,180,tocopilla,-22.0920,-70.1979,19.65,68,31,4.10,CL,1695839018
193,193,fredericton,45.9454,-66.6656,17.13,67,20,0.00,CA,1695839020
278,278,chernomorskoye,45.5039,32.6997,21.31,49,14,7.59,UA,1695839030


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_destinations_df[["City", "Country", "Lat", "Lng", "Humidity"]])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,fallon,US,39.4735,-118.7774,37,
40,villa gesell,AR,-37.2639,-56.9730,59,
45,kapuskasing,CA,49.4169,-82.4331,42,
51,constantia,RO,44.1833,28.6500,65,
81,guerrero negro,MX,27.9769,-114.0611,61,
104,kununurra,AU,-15.7667,128.7333,60,
137,dryden,CA,49.7833,-92.7503,68,
180,tocopilla,CL,-22.0920,-70.1979,68,
193,fredericton,CA,45.9454,-66.6656,67,
278,chernomorskoye,UA,45.5039,32.6997,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fallon - nearest hotel: Value Inn
villa gesell - nearest hotel: Hotel El Faro
kapuskasing - nearest hotel: Travelodge
constantia - nearest hotel: Hotel Chérica
guerrero negro - nearest hotel: Plaza sal paraiso
kununurra - nearest hotel: Kununurra Country Club Resort
dryden - nearest hotel: Holiday Inn Express
tocopilla - nearest hotel: hotel Isidora
fredericton - nearest hotel: Days Inn
chernomorskoye - nearest hotel: Море
landeck - nearest hotel: Hotel Sonne
san pedro de lloc - nearest hotel: Cesa's Palace
beyneu - nearest hotel: Johan yu
dolores - nearest hotel: Plaza Hotel
taltal - nearest hotel: Hostería Taltal
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
tsiroanomandidy - nearest hotel: Mega Food Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
21,fallon,US,39.4735,-118.7774,37,Value Inn
40,villa gesell,AR,-37.2639,-56.9730,59,Hotel El Faro
45,kapuskasing,CA,49.4169,-82.4331,42,Travelodge
51,constantia,RO,44.1833,28.6500,65,Hotel Chérica
81,guerrero negro,MX,27.9769,-114.0611,61,Plaza sal paraiso
104,kununurra,AU,-15.7667,128.7333,60,Kununurra Country Club Resort
137,dryden,CA,49.7833,-92.7503,68,Holiday Inn Express
180,tocopilla,CL,-22.0920,-70.1979,68,hotel Isidora
193,fredericton,CA,45.9454,-66.6656,67,Days Inn
278,chernomorskoye,UA,45.5039,32.6997,49,Море


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols=['Lng','Lat', 'City', 'Humidity', 'Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)